## 🤖 Ders 3 Uygulaması : Transformer Mimarisi ve İnteraktif Çeviri Simülatörü

Bu proje, ChatGPT ve benzeri Büyük Dil Modellerinin (LLM) temelini oluşturan, 2017 yılında literatüre giren **Transformer** mimarisinin "Tokenizasyon" ve "Çapraz Dikkat (Cross-Attention)" aşamalarını interaktif bir simülasyonla sunmaktadır.

### Uygulamanın Amacı ve Yapay Zeka Felsefesi
Toplumdaki genel kanının aksine, doğal dil işleme (NLP) algoritmaları kelimeleri okumaz veya "anlamaz". Makine öğrenmesi modelleri için metinler, uzaydaki matematiksel koordinatları temsil eden sayılardan (vektörlerden) ibarettir. Bu simülasyon, makinenin insan dilini nasıl sayılara indirgediğini ve bu sayılar üzerinden nasıl mantıksal eşleştirmeler yaptığını kanıtlamak amacıyla tasarlanmıştır.

### Simülatörün Çalışma Mantığı (Adım Adım Pipeline)

Simülatörü çalıştırıp bir Türkçe cümle girdiğinizde, arka planda şu mühendislik adımları sırasıyla gerçekleşir:

1. **Metin Ön İşleme (Text Preprocessing):**
   Kullanıcının girdiği serbest metin, algoritmik hataları önlemek adına standartlaştırılır. Büyük harfler küçültülür ve Türkçe karakterler (ç, ğ, ı, ö, ş, ü) İngilizce standartlarına dönüştürülür.
   
2. **Tokenizasyon (Encoding):**
   Cümledeki her bir kelime, sistemin sözlüğünde (Vocabulary) yer alan benzersiz bir matematiksel kimlik numarasına (Token ID) dönüştürülür. Örneğin, "ben" kelimesi makinenin zihninde `[101]` sayısı olarak kodlanır. Sözlükte olmayan kelimeler ise "Bilinmeyen" anlamına gelen `[999]` (UNK) ID'si ile etiketlenir.

3. **Çapraz Dikkat ve Eşleştirme (Cross-Attention Mapping):**
   Transformer mimarisinin temel mantığıdır. Sistem, elde ettiği Türkçe Token ID dizilimini alır ve İngilizce dil uzayında aynı ID'ye sahip matematiksel karşılıkları bulur. Algoritma bu aşamada kelimelerle değil, tamamen sayılarla (ID) işlem yapar.

4. **İnsan Diline Dönüş (Decoding):**
   Eşleşmesi tamamlanan ve hedef dile (İngilizceye) aktarılan sayılar, "Detokenization" işlemi ile yeniden okunabilir metin formatına çevrilerek kullanıcıya nihai çıktı olarak sunulur.

### Çıkarım
Bu interaktif araç; "Yapay zeka nasıl düşünür?" sorusunun felsefi değil, tamamen matematiksel ve istatistiksel bir cevabıdır. Kod çalıştırıldığında, arka planda dönen sayısal dönüşüm matrisleri doğrudan terminal ekranından izlenebilir.

### ! Not: Simülasyon ve Gerçek Dünya Sistemleri Arasındaki Farklar

Bu uygulamada geliştirilen "Mini Transformer Simülatörü", modern Büyük Dil Modellerinin (LLM) kaputunun altında yatan veri işleme felsefesini (Tokenizasyon ve Çapraz Dikkat) kavramsal olarak kusursuz bir şekilde modeller. Ancak, bu kod mimarisi eğitim amaçlı tasarlanmış bir "masaüstü maketi"dir. 

Gerçek hayatta endüstriyel ölçekte çalışan sistemler (Örn: GPT-4, Gemini) ile bu simülasyon arasında devasa matematiksel ve mimari farklar bulunur:

* **Birebir Eşleşme (1:1 Mapping) vs. Bağlamsal Dikkat (Contextual Attention):**
    Simülatörümüz kelimeleri sabit sözlüklerle doğrudan birbirine bağlar. Gerçek bir Transformer modeli ise kelimeleri sabit sözlüklerle çevirmez; "Sorgu, Anahtar ve Değer" (Query, Key, Value) adı verilen devasa matris çarpımları yaparak kelimenin o cümledeki "bağlamına" göre dinamik olarak karar verir.
* **Kelime Bazlı vs. Alt-Kelime (BPE) Tokenizasyon:**
    Simülatörümüz kelimeleri bütün olarak (boşluklardan bölerek) okur ve sözlükte yoksa "Bilinmeyen" (UNK) hatası verir. Gerçek sistemler ise kelimeleri hecelerine veya karakter gruplarına (Byte Pair Encoding - BPE) böler. Bu sayede model, daha önce hiç görmediği bir kelimeyi bile parçalarına ayırarak işleyebilir.
* **Doğrudan Çeviri vs. Diziden Diziye (Seq2Seq) Üretim:**
    Simülatörümüz Türkçe kelime sırasını bozmadan doğrudan İngilizce karşılıklarını yan yana dizer. Gerçek modeller ise çıktı üretirken (Decoding) hedef dilin gramer kurallarını (Örn: İngilizcede yüklemin ortada olması) matematiksel olasılıklarla hesaplayıp kelimelerin yerlerini otomatik olarak yeniden düzenler.
* **Ölçek ve Parametre Hacmi:**
    Simülatörümüz yaklaşık 300 kelimelik bir sözlükle milisaniyeler içinde çalışır. Modern dil modelleri ise 100.000'den fazla benzersiz token içeren sözlüklere sahiptir ve milyarlarca parametre (ağırlık) üzerinden devasa GPU kümelerinde eğitilir.

**Özetle:** Bu Simülatör, devasa bir uçağın çalışma prensibini gösteren aerodinamik bir makettir. Temel fizik kuralları (Tokenizasyon ve ID Eşleştirme) gerçektir, ancak motor gücü ve matematiksel karmaşıklık pekiştirme ve eğitim amacıyla basitleştirilmiştir.

In [ ]:
import time
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

print("INTERAKTIF TRANSFORMER: DIL ISLEME VE CEVIRI SIMULATORU \n")

# --- 1. SOZLUK (VOCABULARY) VERITABANI (300+ Kelime) ---
tr_vocab = {
    # 1. Zamirler ve Soru Kelimeleri
    "ben": 101, "sen": 102, "o": 103, "biz": 104, "siz": 105, "onlar": 106, 
    "bu": 107, "su": 108, "bunlar": 109, "sunlar": 110, "hepsi": 111, "hicbiri": 112, 
    "kim": 113, "ne": 114, "nerede": 115, "nasil": 116, "neden": 117, "nicin": 118, 
    "ne_zaman": 119, "hangi": 120, "kac": 121, "kimin": 122, "nereye": 123, "nereden": 124,

    # 2. Teknoloji, Yapay Zeka ve Bilim
    "yapay": 201, "zeka": 202, "makine": 203, "ogrenmesi": 204, "dogal": 205, "dil": 206, 
    "veri": 207, "kod": 208, "python": 209, "model": 210, "matematik": 211, "algoritma": 212, 
    "bilgisayar": 213, "yazilim": 214, "donanim": 215, "ag": 216, "sistem": 217, "program": 218, 
    "dosya": 219, "ekran": 220, "klavye": 221, "fare": 222, "internet": 223, "teknoloji": 224, 
    "bilgi": 225, "islem": 226, "sunucu": 227, "bulut": 228, "robot": 229, "otonom": 230, 
    "derin": 231, "sinir": 232, "aglari": 233, "analiz": 234, "istatistik": 235, "bilim": 236, 
    "dijital": 237, "sanal": 238, "gerceklik": 239,

    # 3. Zaman ve Mekan
    "simdi": 301, "sonra": 302, "once": 303, "bugun": 304, "yarin": 305, "dun": 306, 
    "sabah": 307, "ogle": 308, "aksam": 309, "gece": 310, "burada": 311, "surada": 312, 
    "orada": 313, "iceri": 314, "disari": 315, "uzak": 316, "yakin": 317, "erken": 318, 
    "gec": 319, "her_zaman": 320, "bazen": 321, "asla": 322, "artik": 323, "henuz": 324, 
    "aniden": 325, "hemen": 326, "her_yer": 327, "hicbir_yer": 328,

    # 4. Isimler (Genel)
    "insan": 401, "adam": 402, "kadin": 403, "cocuk": 404, "aile": 405, "anne": 406, "baba": 407, 
    "arkadas": 408, "ev": 409, "okul": 410, "is": 411, "ofis": 412, "araba": 413, "yol": 414, 
    "sehir": 415, "ulke": 416, "dunya": 417, "hayat": 418, "zaman": 419, "gun": 420, "hafta": 421, 
    "ay": 422, "yil": 423, "su": 424, "yemek": 425, "kitap": 426, "masa": 427, "sandalye": 428, 
    "kalem": 429, "kapi": 430, "pencere": 431, "gunes": 432, "yildiz": 433, "hava": 434, 
    "deniz": 435, "agac": 436, "orman": 437, "hayvan": 438, "kedi": 439, "kopek": 440, "kus": 441, 
    "balik": 442, "para": 443, "sorun": 444, "cevap": 445, "isim": 446, "muzik": 447, "oyun": 448, 
    "film": 449, "resim": 450, "kalp": 451, "akil": 452, "ruh": 453, "ruya": 454, "gercek": 455, 
    "yalan": 456, "sarki": 457, "hikaye": 458, "tarih": 459, "sanat": 460, "toplum": 461, 
    "enerji": 462, "guc": 463, "fikir": 464, "duygu": 465, "umut": 466, "korku": 467, "sevgi": 468, 
    "baris": 469, "savas": 470, "basari": 471, "hata": 472,

    # 5. Fiiller (Simdiki Zaman - 1. Tekil Sahis & Kokler)
    "yapiyorum": 501, "ediyorum": 502, "gidiyorum": 503, "geliyorum": 504, "istiyorum": 505, 
    "biliyorum": 506, "dusunuyorum": 507, "anliyorum": 508, "calisiyorum": 509, "okuyorum": 510, 
    "yaziyorum": 511, "goruyorum": 512, "duyuyorum": 513, "izliyorum": 514, "seviyorum": 515, 
    "bekliyorum": 516, "soruyorum": 517, "ogreniyorum": 518, "kullaniyorum": 519, "konusuyorum": 520, 
    "dinliyorum": 521, "hissediyorum": 522, "basliyorum": 523, "bitiriyorum": 524, "aciyorum": 525, 
    "kapatiyorum": 526, "buluyorum": 527, "kaybediyorum": 528, "kazaniyorum": 529, "deniyorum": 530, 
    "cevapliyorum": 531, "ogretiyorum": 532, "yasiyorum": 533, "yiyorum": 534, "iciyorum": 535, 
    "uyuyorum": 536, "uyaniyorum": 537, "duruyorum": 538, "kosuyorum": 539, "yuruyorum": 540, 
    "oynuyorum": 541, "aliyorum": 542, "veriyorum": 543, "anlatiyorum": 544, "hatirliyorum": 545, 
    "unutuyorum": 546, "bakiyorum": 547, "inaniyorum": 548, "degisiyorum": 549, "uretiyorum": 550,

    # 6. Sifatlar ve Zarflar
    "iyi": 601, "kotu": 602, "guzel": 603, "cirkin": 604, "buyuk": 605, "kucuk": 606, 
    "uzun": 607, "kisa": 608, "sicak": 609, "soguk": 610, "yeni": 611, "eski": 612, 
    "genc": 613, "yasli": 614, "dogru": 615, "yanlis": 616, "hizli": 617, "yavas": 618, 
    "kolay": 619, "zor": 620, "ucuz": 621, "pahali": 622, "mutlu": 623, "uzgun": 624, 
    "guclu": 625, "zayif": 626, "temiz": 627, "kirli": 628, "dolu": 629, "bos": 630, 
    "acik": 631, "kapali": 632, "zengin": 633, "fakir": 634, "agir": 635, "hafif": 636, 
    "derin": 637, "sig": 638, "sert": 639, "yumusak": 640, "parlak": 641, "karanlik": 642, 
    "cesur": 643, "korkak": 644, "akilli": 645, "aptal": 646, "onemli": 647, "gereksiz": 648, 
    "farkli": 649, "ayni": 650,

    # 7. Renkler, Sayilar ve Miktarlar
    "kirmizi": 701, "mavi": 702, "yesil": 703, "sari": 704, "siyah": 705, "beyaz": 706, 
    "gri": 707, "mor": 708, "turuncu": 709, "kahverengi": 710, "pembe": 711, 
    "bir": 712, "iki": 713, "uc": 714, "dort": 715, "bes": 716, "alti": 717, "yedi": 718, 
    "sekiz": 719, "dokuz": 720, "on": 721, "yuz": 722, "bin": 723, 
    "cok": 724, "az": 725, "biraz": 726, "fazla": 727, "butun": 728, "bazi": 729, "hic": 730,

    # 8. Baglaclar ve Edatlar (Genel)
    "ve": 801, "veya": 802, "ama": 803, "fakat": 804, "icin": 805, "cunku": 806, 
    "ile": 807, "gibi": 808, "kadar": 809, "eger": 810, "ragmen": 811, 
    "merhaba": 812, "evet": 813, "hayir": 814, "lutfen": 815, "tesekkurler": 816, 
    "tamam": 817, "selam": 818, "hosca_kal": 819, "gunaydin": 820, "iyi_geceler": 821, 
    "belki": 822, "kesinlikle": 823, "genellikle": 824, "tabii_ki": 825
}

en_vocab = {
    # 1. Pronouns & Questions
    101: "I", 102: "you", 103: "he/she/it", 104: "we", 105: "you all", 106: "they", 
    107: "this", 108: "that", 109: "these", 110: "those", 111: "all", 112: "none", 
    113: "who", 114: "what", 115: "where", 116: "how", 117: "why", 118: "why", 
    119: "when", 120: "which", 121: "how many", 122: "whose", 123: "to where", 124: "from where",

    # 2. Tech & Science
    201: "artificial", 202: "intelligence", 203: "machine", 204: "learning", 205: "natural", 206: "language", 
    207: "data", 208: "code", 209: "python", 210: "model", 211: "math", 212: "algorithm", 
    213: "computer", 214: "software", 215: "hardware", 216: "network", 217: "system", 218: "program", 
    219: "file", 220: "screen", 221: "keyboard", 222: "mouse", 223: "internet", 224: "technology", 
    225: "information", 226: "process", 227: "server", 228: "cloud", 229: "robot", 230: "autonomous", 
    231: "deep", 232: "neural", 233: "networks", 234: "analysis", 235: "statistics", 236: "science", 
    237: "digital", 238: "virtual", 239: "reality",

    # 3. Time & Place
    301: "now", 302: "later", 303: "before", 304: "today", 305: "tomorrow", 306: "yesterday", 
    307: "morning", 308: "noon", 309: "evening", 310: "night", 311: "here", 312: "there", 
    313: "over there", 314: "inside", 315: "outside", 316: "far", 317: "near", 318: "early", 
    319: "late", 320: "always", 321: "sometimes", 322: "never", 323: "anymore", 324: "yet", 
    325: "suddenly", 326: "immediately", 327: "everywhere", 328: "nowhere",

    # 4. Nouns
    401: "human", 402: "man", 403: "woman", 404: "child", 405: "family", 406: "mother", 407: "father", 
    408: "friend", 409: "home", 410: "school", 411: "job", 412: "office", 413: "car", 414: "road", 
    415: "city", 416: "country", 417: "world", 418: "life", 419: "time", 420: "day", 421: "week", 
    422: "month", 423: "year", 424: "water", 425: "food", 426: "book", 427: "table", 428: "chair", 
    429: "pen", 430: "door", 431: "window", 432: "sun", 433: "star", 434: "weather", 
    435: "sea", 436: "tree", 437: "forest", 438: "animal", 439: "cat", 440: "dog", 441: "bird", 
    442: "fish", 443: "money", 444: "problem", 445: "answer", 446: "name", 447: "music", 448: "game", 
    449: "movie", 450: "picture", 451: "heart", 452: "mind", 453: "soul", 454: "dream", 455: "truth", 
    456: "lie", 457: "song", 458: "story", 459: "history", 460: "art", 461: "society", 
    462: "energy", 463: "power", 464: "idea", 465: "emotion", 466: "hope", 467: "fear", 468: "love", 
    469: "peace", 470: "war", 471: "success", 472: "mistake",

    # 5. Verbs (Continuous/Base matching)
    501: "am doing", 502: "am making", 503: "am going", 504: "am coming", 505: "want", 
    506: "know", 507: "am thinking", 508: "understand", 509: "am working", 510: "am reading", 
    511: "am writing", 512: "see", 513: "hear", 514: "am watching", 515: "love", 
    516: "am waiting", 517: "am asking", 518: "am learning", 519: "am using", 520: "am speaking", 
    521: "am listening", 522: "feel", 523: "am starting", 524: "am finishing", 525: "am opening", 
    526: "am closing", 527: "am finding", 528: "am losing", 529: "am winning", 530: "am trying", 
    531: "am answering", 532: "am teaching", 533: "am living", 534: "am eating", 535: "am drinking", 
    536: "am sleeping", 537: "am waking up", 538: "am standing", 539: "am running", 540: "am walking", 
    541: "am playing", 542: "am taking", 543: "am giving", 544: "am explaining", 545: "remember", 
    546: "forget", 547: "am looking", 548: "believe", 549: "am changing", 550: "am producing",

    # 6. Adjectives & Adverbs
    601: "good", 602: "bad", 603: "beautiful", 604: "ugly", 605: "big", 606: "small", 
    607: "long", 608: "short", 609: "hot", 610: "cold", 611: "new", 612: "old", 
    613: "young", 614: "old", 615: "right", 616: "wrong", 617: "fast", 618: "slow", 
    619: "easy", 620: "hard", 621: "cheap", 622: "expensive", 623: "happy", 624: "sad", 
    625: "strong", 626: "weak", 627: "clean", 628: "dirty", 629: "full", 630: "empty", 
    631: "open", 632: "closed", 633: "rich", 634: "poor", 635: "heavy", 636: "light", 
    637: "deep", 638: "shallow", 639: "hard", 640: "soft", 641: "bright", 642: "dark", 
    643: "brave", 644: "coward", 645: "smart", 646: "stupid", 647: "important", 648: "unnecessary", 
    649: "different", 650: "same",

    # 7. Colors, Numbers & Quantifiers
    701: "red", 702: "blue", 703: "green", 704: "yellow", 705: "black", 706: "white", 
    707: "gray", 708: "purple", 709: "orange", 710: "brown", 711: "pink", 
    712: "one", 713: "two", 714: "three", 715: "four", 716: "five", 717: "six", 718: "seven", 
    719: "eight", 720: "nine", 721: "ten", 722: "hundred", 723: "thousand", 
    724: "very/a lot", 725: "little", 726: "some", 727: "too much", 728: "whole", 729: "some", 730: "none",

    # 8. Conjunctions & Particles
    801: "and", 802: "or", 803: "but", 804: "however", 805: "for", 806: "because", 
    807: "with", 808: "like", 809: "until/as much as", 810: "if", 811: "despite", 
    812: "hello", 813: "yes", 814: "no", 815: "please", 816: "thanks", 
    817: "okay", 818: "hi", 819: "goodbye", 820: "good morning", 821: "good night", 
    822: "maybe", 823: "definitely", 824: "usually", 825: "of course"
}

def metni_temizle(metin):
    degisimler = str.maketrans("çğıöşüÇĞİÖŞÜ", "cgiosuCGIOSU")
    return metin.translate(degisimler).lower()

def tokenize_et(metin):
    temiz_metin = metni_temizle(metin)
    kelimeler = temiz_metin.split()
    tokens = []
    for kelime in kelimeler:
        token_id = tr_vocab.get(kelime, 999) 
        tokens.append((kelime, token_id))
    return tokens

def gorsellestir(tr_tokens, en_kelimeler):
    boyut = max(4, len(tr_tokens) * 1.2)
    fig, ax = plt.subplots(figsize=(10, boyut))
    ax.axis('off')
    
    plt.title("Capraz Dikkat (Cross-Attention) ve Token Eslesmesi", fontsize=14, pad=20)
    
    # Basliklar
    ax.text(0.15, len(tr_tokens) + 0.5, "Turkce Girdi", ha='center', fontweight='bold', fontsize=11)
    ax.text(0.50, len(tr_tokens) + 0.5, "Token ID (Makine Dili)", ha='center', fontweight='bold', fontsize=11)
    ax.text(0.85, len(tr_tokens) + 0.5, "Ingilizce Cikti", ha='center', fontweight='bold', fontsize=11)

    for i, (tr_word, t_id) in enumerate(tr_tokens):
        y = len(tr_tokens) - i 
        en_word = en_kelimeler[i]
        
        # Kutular
        ax.text(0.15, y, tr_word, ha='center', va='center', fontsize=12, 
                bbox=dict(boxstyle="round,pad=0.5", facecolor="#e0f7fa", edgecolor="#00bcd4"))
        
        ax.text(0.50, y, f"[{t_id}]", ha='center', va='center', fontsize=11, fontweight='bold',
                bbox=dict(boxstyle="square,pad=0.4", facecolor="#fff9c4", edgecolor="#fbc02d"))
        
        ax.text(0.85, y, en_word, ha='center', va='center', fontsize=12, 
                bbox=dict(boxstyle="round,pad=0.5", facecolor="#e8f5e9", edgecolor="#4caf50"))
        
        # Oklari Ciz (TR -> ID -> EN)
        ax.annotate("", xy=(0.42, y), xytext=(0.25, y), arrowprops=dict(arrowstyle="->", color="gray", lw=1.5))
        ax.annotate("", xy=(0.75, y), xytext=(0.58, y), arrowprops=dict(arrowstyle="->", color="gray", lw=1.5))

    plt.tight_layout()
    plt.show()

# --- 2. KULLANICI ETKILEISIMI ---
print("Sistem Hazir. Genisletilmis sozluk devrede.")
print("Ornek kelimeler: ben, simdi, kod, yaziyorum, dunya, nerede, neden, anliyorum...\n")
kullanici_metni = input("Lutfen cevrilecek Turkce bir cumle girin: ")

if not kullanici_metni.strip():
    kullanici_metni = "ben python ogreniyorum ve makine anliyorum"
    print(f"\nBos giris yapildi. Varsayilan test cumlesi kullaniliyor: '{kullanici_metni}'")

# --- 3. ASAMA: TOKENIZASYON VE CEVIRI ISLEMLERI ---
print("\nASAMA 1: TOKENIZASYON (Makinenin Anlama Sureci)")
time.sleep(0.5)

tr_tokens = tokenize_et(kullanici_metni)
girdi_vektoru = [t[1] for t in tr_tokens]

print("Yapay zeka cumleyi parcaliyor ve kelimeleri Token ID'lerine donusturuyor:")
for kelime, token_id in tr_tokens:
    durum = "(Bilinmeyen Kelime - UNK)" if token_id == 999 else ""
    print(f"   Kelime: '{kelime:<12}' -> Token ID: [{token_id}] {durum}")

print(f"\nSistemin Isleme Aldigi Saf Vektor Dizilimi: {girdi_vektoru}")

print("\nASAMA 2: CAPRAZ DIKKAT VE ESLESTIRME (Cross-Attention)")
time.sleep(0.5)

print("Sistem sayi dizilimini alip Ingilizce uzayindaki karsiliklariyla eslestiriyor. (Gorsel aciliyor...)")

ingilizce_cumle = []
for kelime, token_id in tr_tokens:
    if token_id == 999:
        ingilizce_cumle.append("[UNKNOWN]")
    else:
        ingilizce_cumle.append(en_vocab[token_id])

# Gorsellestirme fonksiyonunu cagiriyoruz
gorsellestir(tr_tokens, ingilizce_cumle)

print("\nASAMA 3: INSAN DILINE DONUS (Decoding)")
time.sleep(0.5)

final_metin = " ".join(ingilizce_cumle)
print("Bulunan Ingilizce tokenlar tekrar anlamli bir metne birlestiriliyor...\n")
print(f"CEVIRI SONUCU: '{final_metin.capitalize()}'\n")

### Kendi Kelimelerini Ekle: Sözlük (Vocabulary) Nasıl Genişletilir?

Sistemin çeviri yeteneğini kendi cümlelerinize göre uyarlamak çok kolaydır. Yeni kelime eklerken şu iki kurala dikkat etmeniz yeterlidir:

1. **Aynı ID Numarasını Kullanın:** Makine kelimeleri değil, sayıları (Token ID) eşleştirir. Türkçe ve İngilizce sözlüğe eklediğiniz kelimelerin kimlik numarası birebir aynı olmalıdır.
2. **Türkçe Karakter Kullanmayın:** Kodun veriyi hatasız okuyabilmesi için `tr_vocab` sözlüğüne kelime eklerken tamamen küçük harf kullanın ve Türkçe karakter (ç, ğ, ı, ö, ş, ü) **içermemesine** dikkat edin. (İçerse bile sorun çıkmaması için düzenleme yapıldı yine de gelecekteki projeler için dikkat edilmeli)

**Örnek Ekleme:**
Diyelim ki sisteme "mühendis" kelimesini eklemek istiyoruz ve boşta olan `901` ID'sini seçtik:

```python
# 1. Türkçe sözlüğe (tr_vocab) ekle:
"muhendis": 901,  # 'ü' harfi yerine 'u' kullanıldı

# 2. İngilizce sözlüğe (en_vocab) AYNI ID ile ekle:
901: "engineer",